# Training on Kaggle Dataset "Car Parts and Car Damages"

In [1]:
from ultralytics import YOLO
import json
import numpy as np
from matplotlib import pyplot as plt
import os
import shutil
import torch

In [3]:
dataloc = "../carinsuranceData/Car damages dataset/File1/img"
annloc = "../carinsuranceData/Car damages dataset/File1/ann"

## Utilities

In [4]:
def load_jsons(json_name):
    with open(json_name,"r") as json_file:
        train_json_handler = json.load(json_file)
        return train_json_handler

def getJson(path):
    with open(path,"r") as jsonfile:
        jsonloader = json.load(jsonfile)
        return jsonloader

categories = []

def save_classes(class_):
    if class_ in categories:
        return categories.index(class_)
    else:
        categories.append(class_)
        return len(categories)-1

def getImageAnnotations(json_ann):
    objects = json_ann["objects"]
    width,height = json_ann["size"]["width"],json_ann["size"]["height"]
    catlist = []
    for cat in objects:
        catname = cat["classTitle"]
        catindex = save_classes(catname)
        points = [str(catindex),]
        for x,y in cat["points"]["exterior"]:
            points.append(str(x/width)),points.append(str(y/height))
        catlist.append(points)
    return catlist,width,height


def createtxt(filename, catlist,newpath):
    filename = filename.split(".")[0]
    with open(newpath+"/"+filename+".txt","w") as filep:
        for i in catlist:
            filep.write(" ".join(i))
            filep.write("\n")


def parseFolderAnnots(path):
    #send every 3rd image in val data
    counter = 0
    for annot in os.listdir(os.path.join(os.getcwd(),path+r"\ann")):
        if counter%3 == 0:
            newp = "CarDamageDataset/val"

        else:
            newp = "CarDamageDataset/train"

        
        image_name = annot.replace(".json","")
        counter+=1
        jsonp = os.path.join(os.getcwd(),path+r"\ann",annot)
        
        json_ann = load_jsons(jsonp)
        catlist,width,height = getImageAnnotations(json_ann)
        
        shutil.copy(os.path.join(os.getcwd(),path+r"\img",image_name),newp+"/images/"+image_name )


        createtxt(annot,catlist,newp+"/labels")
    print(categories)
        
        


In [5]:
parseFolderAnnots(path = r"carinsuranceData\Car damages dataset\File1")

['Back-bumper', 'Back-door', 'Back-wheel', 'Back-window', 'Back-windshield', 'Fender', 'Front-bumper', 'Front-door', 'Front-wheel', 'Front-window', 'Headlight', 'Hood', 'Mirror', 'Quarter-panel', 'Rocker-panel', 'Roof', 'Tail-light', 'Trunk', 'Windshield', 'Grille', 'License-plate']


In [6]:
model = YOLO("yolo11n-seg.pt") 
torch.cuda.empty_cache()
results = model.train(
    data = "CarDamageDataset/data.yaml",
  epochs=400,
)

Ultralytics 8.3.40  Python-3.11.0rc2 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=CarDamageDataset/data.yaml, epochs=400, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

train: Scanning C:\Projects\InsuranceAutomation\CarDamageDataset\train\labels.cache... 665 images, 0 backgrounds, 0 corrupt: 100%|██████████| 665/665 [00:00<?, ?it/s]
val: Scanning C:\Projects\InsuranceAutomation\CarDamageDataset\val\labels.cache... 333 images, 0 backgrounds, 0 corrupt: 100%|██████████| 333/333 [00:00<?, ?it/s]


Plotting labels to runs\segment\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.0004, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\segment\train6
Starting training for 400 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/400      4.51G      1.544      3.196      4.324      1.523        240        640: 100%|██████████| 42/42 [00:15<00:00,  2.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.50it/s]

                   all        333       5366       0.02     0.0226     0.0165     0.0124     0.0198     0.0223     0.0163     0.0112


...

    400/400      3.54G     0.5148     0.7688     0.3607     0.8771        139        640: 100%|██████████| 42/42 [00:10<00:00,  4.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  3.66it/s]

                   all        333       5366      0.882      0.847      0.877      0.645      0.897      0.848      0.879      0.604



400 epochs completed in 2.812 hours.
Optimizer stripped from runs\segment\train6\weights\last.pt, 6.1MB
Optimizer stripped from runs\segment\train6\weights\best.pt, 6.1MB

Validating runs\segment\train6\weights\best.pt...
Ultralytics 8.3.40  Python-3.11.0rc2 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
YOLO11n-seg summary (fused): 265 layers, 2,838,663 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:04<00:00,  2.55it/s]


                   all        333       5366      0.888      0.849      0.882      0.656      0.899      0.851      0.885      0.616
           Back-bumper        159        160      0.912      0.831      0.881      0.677      0.902      0.819      0.849      0.661
             Back-door        249        253      0.931      0.941      0.966      0.843      0.929      0.937      0.956      0.681
            Back-wheel        298        302      0.942      0.957      0.966      0.775       0.95      0.964       0.97      0.799
           Back-window        284        433      0.879      0.882      0.912      0.604       0.88      0.875      0.895      0.583
       Back-windshield         97         97      0.967      0.918       0.95      0.787      0.967      0.914      0.941      0.762
                Fender        299        307      0.922      0.899      0.935      0.718      0.932      0.906      0.931      0.684
          Front-bumper        225        227      0.933      0.903   